# PreProcess 1 file 

In [32]:
import os
import re


directory_path = 'data/archive'

# Compile the regex pattern to extract wins or losses for IlxxxlI
pattern = re.compile(r"Player IlxxxlI.*?(Wins|Loses): (\d+\.\d+)")

# Initialize total wins and losses
total_wins = 0.0
total_losses = 0.0
hand_played=0

# Iterate through all files in the directory
for filename in os.listdir(directory_path):
    
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        # Read the log data from the file
        with open(file_path, 'r') as file:
            log_data = file.read()
        
        # Find all matches of the pattern in the log data
        matches = pattern.findall(log_data)
        hand_played= hand_played+len(matches)
        print(filename,len(matches))
        
        # Sum up the wins and losses
        for match in matches:
            action, amount = match
            amount = float(amount)
            if action == 'Wins':
                total_wins += amount
            elif action == 'Loses':
                total_losses += amount
    

# Output the total wins and losses
print(f"Total wins of IlxxxlI: {total_wins}")
print(f"Total losses of IlxxxlI: {total_losses}")
print(f"Total games of IlxxxlI: {hand_played}")


File204.txt 394
File199.txt 122
File198.txt 150
File201.txt 73
File200.txt 109
Export Holdem Manager 2.0 12302016144830.txt 17307
File203.txt 285
File196.txt 122
Export Holdem Manager 2.0 12292016131233.txt 850
Total wins of IlxxxlI: 25303.470000000354
Total losses of IlxxxlI: 19029.1700000005
Total games of IlxxxlI: 19412


In [23]:
pattern = re.compile(r"Game started at")

# Initialize the game count
total_games = 0

# Iterate through all files in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        # Read the log data from the file
        with open(file_path, 'r') as file:
            log_data = file.read()
        
        # Count the occurrences of the pattern in the log data
        games_in_file = len(pattern.findall(log_data))
        total_games += games_in_file

# Output the total number of games
print(f"Total number of games: {total_games}")

Total number of games: 52156


In [33]:
# Compile the regex pattern to match lines ending with "received a card."
path = 'data/archive'
pattern = re.compile(r".*received a card\.\n")

# Iterate through all files in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        # Read the log data from the file
        with open(file_path, 'r') as file:
            log_data = file.readlines()
        
        # Filter out lines that match the pattern
        cleaned_data = [line for line in log_data if not pattern.match(line)]
        
        # Write the cleaned data back to the file
        with open(file_path, 'w') as file:
            file.writelines(cleaned_data)

In [36]:
import os
import re

# Directory containing the log files
directory_path = 'data/archive'

# Compile the regex pattern to match and replace the "Game started at" line
pattern = re.compile(r"Game started at: .*")

# Iterate through all files in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        # Read the log data from the file
        with open(file_path, 'r') as file:
            log_data = file.readlines()
        
        # Process each line and modify if necessary
        modified_data = []
        for line in log_data:
            modified_line = pattern.sub("Game started", line)
            modified_data.append(modified_line)
        
        # Write the modified data back to the same file
        with open(file_path, 'w') as file:
            file.writelines(modified_data)

print("Modified all lines to keep only 'Game started'.")


Modified all lines to keep only 'Game started'.


In [44]:
import os
import re

# Directory containing the log files
directory_path = 'data/archive'

# Regex patterns to identify game boundaries and summaries
game_start_pattern = re.compile(r"Game started")
summary_start_pattern = re.compile(r"------ Summary ------")
summary_end_pattern = re.compile(r"Game ended at:.*")

# Function to check if IlxxxlI lost money in the summary
def player_lost_in_summary(summary):
    lose_pattern = re.compile(r"Player IlxxxlI.*Loses:")
    return lose_pattern.search(summary) is not None

# Iterate through all files in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        # Read the log data from the file
        with open(file_path, 'r') as file:
            log_data = file.read()
        
        # Split the log data into individual games
        games = re.split(game_start_pattern, log_data)
        
        # The first element will be empty if the file starts with "Game started at:", so we discard it
        if not games[0].strip():
            games = games[1:]
        
        # Filter the games
        filtered_games = []
        for game in games:
            game_content = "Game started:" + game
            summary_start = summary_start_pattern.search(game_content)
            if summary_start:
                if not player_lost_in_summary(game_content):
                    filtered_games.append(game_content)
        
        # Write the filtered games back to the same file
        with open(file_path+'test', 'w') as file:
            file.writelines(filtered_games)

print("Filtered games where player IlxxxlI does not lose money.")


Filtered games where player IlxxxlI does not lose money.


In [43]:
import os
import re

# Directory containing the log files
directory_path = 'data/treated'

# Compile regex patterns to match seat lines and player names in actions
seat_pattern = re.compile(r"Seat (\d+): (\w+) \(([\d\.]+)\)\.")
action_pattern = re.compile(r"Player (\w+)")

# Function to replace player names according to seat numbers
def replace_player_names(text):
    # Find all seat assignments
    seats = seat_pattern.findall(text)
    name_map = {}
    for seat in seats:
        seat_number, player_name, balance = seat
        if player_name == "IlxxxlI":
            name_map[player_name] = "You"
        else:
            name_map[player_name] = f"Player {seat_number}"
    
    # Replace names in seat assignments
    for player_name, new_name in name_map.items():
        text = re.sub(rf"Seat (\d+): {player_name} \(([\d\.]+)\)\.", rf"Seat \1: {new_name} (\2).", text)
    
    # Replace names in actions
    def replace_action(match):
        player_name = match.group(1)
        return f"Player {name_map.get(player_name, player_name)}"
    
    text = action_pattern.sub(replace_action, text)
    
    return text

# Iterate through all files in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        # Read the log data from the file
        with open(file_path, 'r') as file:
            log_data = file.read()
        
        # Replace player names
        modified_data = replace_player_names(log_data)
        
        # Write the modified data back to the same file
        with open(file_path, 'w') as file:
            file.write(modified_data)

print("Player names have been replaced according to their seat numbers and 'IlxxxlI' has been renamed to 'You'.")


Player names have been replaced according to their seat numbers and 'IlxxxlI' has been renamed to 'You'.


In [50]:
import os
import re
import pandas as pd

# Directory containing the log files
directory_path = 'data/treated'

# Compile the regex patterns
game_start_pattern = re.compile(r"Game started:")
player_action_pattern = re.compile(r"Player IlxxxlI (folds|calls|raises) ?\(?(\d*\.\d*)\)?")
summary_start_pattern = re.compile(r"------ Summary ------")

# Initialize lists to hold the instruction and output data
instructions = []
outputs = []

# Function to extract hand history and action for IlxxxlI
def extract_hand_history_and_action(log_data):
    games = re.split(game_start_pattern, log_data)
    
    # The first element will be empty if the file starts with "Game started:", so we discard it
    if not games[0].strip():
        games = games[1:]
    
    for game in games:
        game_content = "Game started:" + game
        actions = []
        ilxxxlI_action = None
        hand_history = []
        
        # Split the game content into lines
        lines = game_content.split('\n')
        for line in lines:
            if summary_start_pattern.search(line):
                break  # Stop at the summary section
            hand_history.append(line)
            action_match = player_action_pattern.search(line)
            if action_match:
                ilxxxlI_action = action_match.group()
                actions.append(ilxxxlI_action)
        
        if ilxxxlI_action:
            # Join the hand history up to the last action
            instruction = '\n'.join(hand_history)
            # Extract the last action
            output = actions[-1]
            instructions.append(instruction)
            outputs.append(output)

# Iterate through all files in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        # Read the log data from the file
        with open(file_path, 'r') as file:
            log_data = file.read()
        # Extract hand history and action
        extract_hand_history_and_action(log_data)

# Create a DataFrame with the instructions and outputs
df = pd.DataFrame({
    'instruction': instructions,
    'input': '',
    'output': outputs
})

# Save the DataFrame to a CSV file
df.to_csv('poker_dataset.csv', index=False)

print("Dataset created and saved to poker_dataset.csv")


Dataset created and saved to poker_dataset.csv


In [58]:
import os
import re
import pandas as pd

# Directory containing the log files
directory_path = 'data/treated'

# Compile the regex patterns
game_start_pattern = re.compile(r"Game started:")
player_action_pattern = re.compile(r"Player IlxxxlI (folds|calls|raises(?: \((\d+\.\d+|\d+)\))?)")
summary_start_pattern = re.compile(r"------ Summary ------")

# Initialize lists to hold the instruction and output data
instructions = []
outputs = []

# Function to extract hand history and actions for IlxxxlI
def extract_hand_history_and_actions(log_data):
    games = re.split(game_start_pattern, log_data)
    
    # The first element will be empty if the file starts with "Game started:", so we discard it
    if not games[0].strip():
        games = games[1:]
    
    for game in games:
        game_content = "Game started:" + game
        hand_history = []
        
        # Split the game content into lines
        lines = game_content.split('\n')
        for line in lines:
            if summary_start_pattern.search(line):
                break  # Stop at the summary section
            hand_history.append(line)
            action_match = player_action_pattern.search(line)
            if action_match:
                # Join the hand history up to the current action
                instruction = '\n'.join(hand_history[:-1])
                # Extract the action
                output = action_match.group()
                instructions.append(instruction)
                outputs.append(output)

# Iterate through all files in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        # Read the log data from the file
        with open(file_path, 'r') as file:
            log_data = file.read()
        # Extract hand history and actions
        extract_hand_history_and_actions(log_data)

# Create a DataFrame with the instructions and outputs
df = pd.DataFrame({
    'instruction': instructions,
    'input':"",
    'output': outputs
})

# Save the DataFrame to a CSV file
df.to_csv('poker_dataset.csv', index=False)

print("Dataset created and saved to poker_dataset.csv")


Dataset created and saved to poker_dataset.csv


In [59]:
len(df)

29914